# Chapter 07 실습 2: 간단한 GAN 구현

## 실습 목표
- Dense 레이어만으로 구성된 기본 GAN을 처음부터 구현한다
- Generator와 Discriminator를 각각 별도의 옵티마이저로 교대 학습한다
- 30 에포크 동안 학습하며 생성 이미지의 품질 변화를 시각화한다
- DCGAN으로 개선하는 방법을 탐구한다 (도전 과제)

## 실습 개요

```
Dense GAN 구조:

Generator:
  z (100,) → Dense(256, LeakyReLU) → Dense(512, LeakyReLU)
           → Dense(1024, LeakyReLU) → Dense(784, tanh) → 이미지 (28×28)

Discriminator:
  이미지 (784,) → Dense(1024, LeakyReLU) → Dense(512, LeakyReLU)
               → Dense(256, LeakyReLU) → Dense(1, sigmoid) → 진위 확률
```

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib
import time

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f'TensorFlow 버전: {tf.__version__}')

# 재현성 시드
tf.random.set_seed(2024)
np.random.seed(2024)

# 하이퍼파라미터
LATENT_DIM  = 100   # 잠재 벡터 차원 (노이즈 차원)
IMAGE_DIM   = 784   # 28×28 = 784 픽셀
BATCH_SIZE  = 256   # 배치 크기
EPOCHS      = 30    # 학습 에포크 수
LR_G        = 2e-4  # Generator 학습률
LR_D        = 2e-4  # Discriminator 학습률

print(f'잠재 벡터 차원: {LATENT_DIM}')
print(f'이미지 차원:   {IMAGE_DIM} (28×28)')
print(f'배치 크기:     {BATCH_SIZE}')
print(f'학습 에포크:   {EPOCHS}')

In [ ]:
# MNIST 로드 + 정규화
print('MNIST 데이터 로딩...')
(x_train, y_train), (x_test, _) = keras.datasets.mnist.load_data()

# [0, 255] → [-1, 1] 정규화 (Generator의 tanh 출력과 일치)
# tanh: [-1, 1] 출력 → 동일한 범위로 데이터 정규화 필요
x_train = x_train.astype('float32')
x_train = (x_train - 127.5) / 127.5  # 127.5를 빼고 나누면 [-1, 1]

# (60000, 28, 28) → (60000, 784) 평탄화
x_train_flat = x_train.reshape(-1, IMAGE_DIM)

print(f'훈련 데이터 shape: {x_train_flat.shape}')
print(f'픽셀 범위: [{x_train_flat.min():.1f}, {x_train_flat.max():.1f}]')

# TF Dataset 파이프라인
train_dataset = (
    tf.data.Dataset.from_tensor_slices(x_train_flat)
    .shuffle(60000)                          # 전체 데이터 셔플
    .batch(BATCH_SIZE, drop_remainder=True)  # 마지막 불완전 배치 제거
    .prefetch(tf.data.AUTOTUNE)              # 비동기 로딩으로 GPU 대기 시간 감소
)

print(f'배치 수: {len(list(train_dataset))}')

# 원본 이미지 샘플 시각화
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
for i, ax in enumerate(axes.flat):
    digit = i % 10
    idx = np.where(y_train == digit)[0][i // 10]
    # [-1, 1] → [0, 1]로 되돌려 시각화
    img = (x_train_flat[idx] + 1) / 2.0
    ax.imshow(img.reshape(28, 28), cmap='gray')
    ax.set_title(str(digit), fontsize=9)
    ax.axis('off')

plt.suptitle('MNIST 원본 이미지 샘플 (정규화 후 표시)', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
def build_generator(latent_dim=LATENT_DIM, output_dim=IMAGE_DIM):
    """
    Dense 기반 Generator
    노이즈 z (100,) → 이미지 벡터 (784,)
    
    단계별 특징 맵 확장:
    100 → 256 → 512 → 1024 → 784
    """
    model = keras.Sequential([
        # 입력층
        keras.layers.Input(shape=(latent_dim,), name='noise_input'),
        
        # 첫 번째 Dense: 잠재 벡터 확장
        keras.layers.Dense(256),
        keras.layers.BatchNormalization(),   # 학습 안정화
        keras.layers.LeakyReLU(alpha=0.2),  # 음수 기울기 허용
        
        # 두 번째 Dense: 특징 확장
        keras.layers.Dense(512),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # 세 번째 Dense: 특징 확장
        keras.layers.Dense(1024),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # 출력층: 784 픽셀, tanh 활성화 → [-1, 1] 출력
        keras.layers.Dense(output_dim, activation='tanh', name='image_output')
    ], name='Generator')
    
    return model


# Generator 생성 및 확인
generator = build_generator()
generator.summary()

# 동작 테스트
test_noise = tf.random.normal([4, LATENT_DIM])
test_gen_out = generator(test_noise, training=False)
print(f'\n입력 노이즈 shape: {test_noise.shape}')
print(f'생성 이미지 shape: {test_gen_out.shape}')  # (4, 784)
print(f'픽셀 범위: [{test_gen_out.numpy().min():.3f}, {test_gen_out.numpy().max():.3f}]')

# 미학습 Generator의 출력 (노이즈)
fig, axes = plt.subplots(1, 4, figsize=(10, 3))
for i, ax in enumerate(axes):
    img = ((test_gen_out[i].numpy() + 1) / 2.0).reshape(28, 28)
    ax.imshow(img, cmap='gray')
    ax.set_title(f'초기 생성 {i+1}', fontsize=9)
    ax.axis('off')
plt.suptitle('학습 전 Generator 출력 (무작위 노이즈)', fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
def build_discriminator(input_dim=IMAGE_DIM):
    """
    Dense 기반 Discriminator
    이미지 벡터 (784,) → 진위 확률 스칼라
    
    단계별 특징 맵 축소:
    784 → 1024 → 512 → 256 → 1
    """
    model = keras.Sequential([
        # 입력층
        keras.layers.Input(shape=(input_dim,), name='image_input'),
        
        # 첫 번째 Dense: 특징 추출
        keras.layers.Dense(1024),
        keras.layers.LeakyReLU(alpha=0.2),  # 음수 입력에도 기울기 유지
        keras.layers.Dropout(0.3),           # 과적합 방지 (D에서는 BN 대신 Dropout)
        
        # 두 번째 Dense: 특징 압축
        keras.layers.Dense(512),
        keras.layers.LeakyReLU(alpha=0.2),
        keras.layers.Dropout(0.3),
        
        # 세 번째 Dense: 추가 압축
        keras.layers.Dense(256),
        keras.layers.LeakyReLU(alpha=0.2),
        keras.layers.Dropout(0.3),
        
        # 출력층: 진위 확률 [0=가짜, 1=실제]
        keras.layers.Dense(1, activation='sigmoid', name='real_prob')
    ], name='Discriminator')
    
    return model


# Discriminator 생성 및 확인
discriminator = build_discriminator()
discriminator.summary()

# 동작 테스트
test_images = tf.random.normal([4, IMAGE_DIM])
test_disc_out = discriminator(test_images, training=False)
print(f'\n입력 이미지 shape: {test_images.shape}')
print(f'판별 결과 shape:   {test_disc_out.shape}')  # (4, 1)
print(f'판별 확률 (초기 무작위): {test_disc_out.numpy().flatten().round(4)}')

In [ ]:
# 손실 함수
bce = keras.losses.BinaryCrossentropy(from_logits=False)

def generator_loss(fake_output):
    """G 손실: 가짜 이미지를 실제(1)로 속이도록 최소화"""
    return bce(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    """D 손실: 실제=1, 가짜=0으로 분류하도록 최소화"""
    real_loss = bce(tf.ones_like(real_output), real_output)   # 실제 이미지 → 1
    fake_loss = bce(tf.zeros_like(fake_output), fake_output)  # 가짜 이미지 → 0
    return real_loss + fake_loss

# 별도 Optimizer
gen_optimizer  = keras.optimizers.Adam(LR_G, beta_1=0.5)
disc_optimizer = keras.optimizers.Adam(LR_D, beta_1=0.5)

@tf.function  # TF 그래프로 컴파일하여 속도 향상
def train_step(real_images):
    """GAN 한 스텝 학습: D 업데이트 → G 업데이트"""
    batch_size = tf.shape(real_images)[0]
    
    # 노이즈 샘플링
    noise = tf.random.normal([batch_size, LATENT_DIM])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # G 순전파: 가짜 이미지 생성
        fake_images = generator(noise, training=True)
        
        # D 순전파: 실제와 가짜 판별
        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(fake_images, training=True)
        
        # 손실 계산
        g_loss = generator_loss(fake_output)
        d_loss = discriminator_loss(real_output, fake_output)
    
    # D 파라미터 업데이트
    d_grads = disc_tape.gradient(d_loss, discriminator.trainable_variables)
    disc_optimizer.apply_gradients(zip(d_grads, discriminator.trainable_variables))
    
    # G 파라미터 업데이트
    g_grads = gen_tape.gradient(g_loss, generator.trainable_variables)
    gen_optimizer.apply_gradients(zip(g_grads, generator.trainable_variables))
    
    return g_loss, d_loss


# 학습 루프 (30 에포크)
print(f'GAN 학습 시작 ({EPOCHS} 에포크)...')

# 학습 진행 모니터링용 고정 노이즈
monitor_noise = tf.random.normal([16, LATENT_DIM])  # 4×4 격자용 16개

gen_losses  = []
disc_losses = []
epoch_images = []   # 에포크별 생성 이미지 저장

start = time.time()

for epoch in range(EPOCHS):
    epoch_g_losses = []
    epoch_d_losses = []
    
    for batch in train_dataset:
        g_loss, d_loss = train_step(batch)
        epoch_g_losses.append(float(g_loss))
        epoch_d_losses.append(float(d_loss))
    
    # 에포크 평균 손실
    avg_g = np.mean(epoch_g_losses)
    avg_d = np.mean(epoch_d_losses)
    gen_losses.append(avg_g)
    disc_losses.append(avg_d)
    
    # 고정 노이즈로 이미지 생성 저장
    generated = generator(monitor_noise, training=False).numpy()
    epoch_images.append(generated)
    
    elapsed = time.time() - start
    print(f'에포크 {epoch+1:2d}/{EPOCHS} | '
          f'G: {avg_g:.4f} | D: {avg_d:.4f} | '
          f'{elapsed:.1f}초')

print('\n학습 완료!')
print(f'총 소요 시간: {(time.time()-start):.1f}초')

In [ ]:
def plot_grid(images_flat, title, n_rows=4, n_cols=4):
    """
    생성 이미지를 4×4 격자로 시각화
    images_flat: shape = (16, 784), 픽셀 범위 [-1, 1]
    """
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(8, 8))
    for i, ax in enumerate(axes.flat):
        img = ((images_flat[i] + 1) / 2.0).reshape(28, 28)  # [-1,1] → [0,1]
        ax.imshow(img, cmap='gray', vmin=0, vmax=1)
        ax.axis('off')
    plt.suptitle(title, fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.show()


# 에포크 1, 10, 20, 30 비교
milestones = [1, 10, 20, 30]
for ep in milestones:
    idx = min(ep - 1, len(epoch_images) - 1)
    plot_grid(epoch_images[idx], title=f'에포크 {ep} 생성 이미지 (4×4 격자)')

# 손실 곡선
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 손실 그래프
ax1 = axes[0]
ax1.plot(range(1, EPOCHS+1), gen_losses,  'b-o', label='Generator 손실',     linewidth=2, markersize=4)
ax1.plot(range(1, EPOCHS+1), disc_losses, 'r-s', label='Discriminator 손실', linewidth=2, markersize=4)
ax1.axhline(y=np.log(2), color='gray', linestyle='--', alpha=0.8, label=f'균형점 (ln2≈{np.log(2):.3f})')
ax1.set_xlabel('에포크', fontsize=11)
ax1.set_ylabel('손실', fontsize=11)
ax1.set_title('GAN 학습 손실 곡선', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 손실 비율 (G/D 균형 확인)
ax2 = axes[1]
loss_ratio = [g/d if d > 0 else 0 for g, d in zip(gen_losses, disc_losses)]
ax2.plot(range(1, EPOCHS+1), loss_ratio, 'g-^', linewidth=2, markersize=4)
ax2.axhline(y=1.0, color='gray', linestyle='--', alpha=0.8, label='G/D = 1.0 (이상적 균형)')
ax2.set_xlabel('에포크', fontsize=11)
ax2.set_ylabel('G 손실 / D 손실', fontsize=11)
ax2.set_title('G/D 손실 비율 (균형 모니터링)', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.suptitle('Simple GAN 학습 분석', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f'최종 Generator 손실:     {gen_losses[-1]:.4f}')
print(f'최종 Discriminator 손실: {disc_losses[-1]:.4f}')
print(f'이론적 균형점:            {np.log(2):.4f}')

## 도전 과제: DCGAN으로 개선

Dense 기반 GAN을 DCGAN(Deep Convolutional GAN)으로 개선하여 이미지 품질을 향상시켜 보자.

### DCGAN Generator (참고 코드)

```python
def build_dcgan_generator(latent_dim=100):
    model = keras.Sequential([
        keras.layers.Input(shape=(latent_dim,)),
        
        # Dense → Reshape
        keras.layers.Dense(7 * 7 * 256, use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(0.2),
        keras.layers.Reshape((7, 7, 256)),
        
        # Conv2DTranspose 스택
        keras.layers.Conv2DTranspose(128, 5, strides=1, padding='same', use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(0.2),
        
        keras.layers.Conv2DTranspose(64, 5, strides=2, padding='same', use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(0.2),
        
        # 출력층: (28, 28, 1), tanh
        keras.layers.Conv2DTranspose(1, 5, strides=2, padding='same', activation='tanh')
    ], name='DCGAN_Generator')
    return model
```

### DCGAN Discriminator (참고 코드)

```python
def build_dcgan_discriminator():
    model = keras.Sequential([
        keras.layers.Input(shape=(28, 28, 1)),
        
        # Conv2D 스택
        keras.layers.Conv2D(64, 5, strides=2, padding='same'),
        keras.layers.LeakyReLU(0.2),
        keras.layers.Dropout(0.3),
        
        keras.layers.Conv2D(128, 5, strides=2, padding='same'),
        keras.layers.LeakyReLU(0.2),
        keras.layers.Dropout(0.3),
        
        # 분류 헤드
        keras.layers.Flatten(),
        keras.layers.Dense(1, activation='sigmoid')
    ], name='DCGAN_Discriminator')
    return model
```

### Dense GAN vs DCGAN 비교 실험

| 항목 | Dense GAN | DCGAN |
|------|-----------|-------|
| 생성 이미지 품질 | ? | ? |
| 파라미터 수 | ? | ? |
| 학습 시간/에포크 | ? | ? |
| Mode Collapse 빈도 | ? | ? |

### 추가 개선 아이디어

1. **Label Smoothing**: 실제 이미지 레이블 0.9로 설정 (D가 과자신감 방지)
2. **One-sided Label Smoothing**: 실제만 0.9, 가짜는 0 유지
3. **Spectral Normalization**: D의 가중치를 spectral norm으로 제약
4. **WGAN**: BCE 손실 → Wasserstein Distance로 교체
5. **Conditional GAN**: 생성할 숫자 클래스를 조건으로 입력 (c-GAN)

### 참고: Conditional GAN 아이디어

```python
# G 입력: [noise (100,), class_label (10,)] → concatenate
noise_input = keras.layers.Input(shape=(100,))
label_input = keras.layers.Input(shape=(10,))  # one-hot
x = keras.layers.Concatenate()([noise_input, label_input])  # (110,)
# ... Dense 레이어 ...
```